# Bloomsbury Tech Market Analysis

This notebook explores the rich auction dataset provided by the CTO. We will connect securely to the ClickHouse database and perform market analysis on sales, artists, and pricing trends.

In [ ]:
import requests
import pandas as pd
import urllib3
from io import StringIO
import sys
import os

# Add parent directory to path to find db_config
sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('.'))
from db_config import BASE_URL, HEADERS

urllib3.disable_warnings()

def run_query_df(query):
    """Runs a ClickHouse query and returns the result as a Pandas DataFrame."""
    try:
        # Use CSVWithNames format for easy pandas parsing
        full_query = query + " FORMAT CSVWithNames"
        response = requests.post(BASE_URL, headers=HEADERS, data=full_query.encode('utf-8'))
        response.raise_for_status()
        return pd.read_csv(StringIO(response.text))
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

print("Setup complete. Connected to ClickHouse.")

## 1. Top 10 Most Expensive Lots (USD)

We normalize hammer prices to USD using the `fx_rates` table to compare sales across different currencies.

In [ ]:
query_top_sales = """
SELECT DISTINCT
    substring(l.title, 1, 50) as title_short,
    l.creator,
    s.hammer_price,
    s.currency,
    round(s.hammer_price * fx.rate_to_usd, 2) as price_usd,
    toDate(a.date_closed) as sale_date
FROM sothebys.sales s
JOIN sothebys.lots l ON s.lot_uuid = l.lot_uuid
JOIN sothebys.auctions a ON l.auction_uuid = a.auction_uuid
LEFT JOIN sothebys.fx_rates fx ON toDate(a.date_closed) = fx.rate_date AND s.currency = fx.currency
WHERE s.is_sold = 1 AND s.hammer_price > 0 AND fx.rate_to_usd > 0
ORDER BY price_usd DESC
LIMIT 10
"""

df_top_sales = run_query_df(query_top_sales)
display(df_top_sales)

## 2. Top 10 Artists by Total Sales Volume (USD)

Aggregating the total value of sold lots per artist.

In [ ]:
query_top_artists = """
SELECT
    l.creator,
    count() as lots_sold,
    round(sum(s.hammer_price * fx.rate_to_usd), 0) as total_volume_usd
FROM sothebys.sales s
JOIN sothebys.lots l ON s.lot_uuid = l.lot_uuid
JOIN sothebys.auctions a ON l.auction_uuid = a.auction_uuid
LEFT JOIN sothebys.fx_rates fx ON toDate(a.date_closed) = fx.rate_date AND s.currency = fx.currency
WHERE s.is_sold = 1 AND s.hammer_price > 0 AND fx.rate_to_usd > 0 AND l.creator != ''
GROUP BY l.creator
ORDER BY total_volume_usd DESC
LIMIT 10
"""

df_top_artists = run_query_df(query_top_artists)
display(df_top_artists)

## 3. Price vs. Size Correlation

Analyzing the relationship between the surface area of the artwork and its hammer price (log-log scale).

In [ ]:
query_corr = """
SELECT
    corr(log_hammer_price, log_surface_area) as correlation_coefficient,
    count() as sample_size
FROM sothebys.gold_features
WHERE log_hammer_price IS NOT NULL AND log_surface_area IS NOT NULL
"""

df_corr = run_query_df(query_corr)
display(df_corr)